Hinge Loss
Used for: SVM classification, maximum-margin classification

Binary Hinge Loss:


def hinge_loss(y_true, y_pred):
    """y_true ∈ {-1, 1}"""
    return np.maximum(0, 1 - y_true * y_pred)

# PyTorch implementation
class HingeLoss(nn.Module):
    def __init__(self, margin=1.0):
        super().__init__()
        self.margin = margin
    
    def forward(self, y_pred, y_true):
        # y_true: 1 or -1
        loss = torch.clamp(self.margin - y_true * y_pred, min=0)
        return loss.mean()
Multi-class Hinge Loss (Crammer-Singer):


def multiclass_hinge_loss(y_true, y_pred):
    """
    y_true: one-hot encoded or class indices
    y_pred: raw scores/logits
    """
    correct_class_score = y_pred[y_true]
    margins = y_pred - correct_class_score[:, np.newaxis] + 1
    margins[np.arange(len(y_true)), y_true] = 0
    return np.maximum(0, margins).sum(axis=1).mean()
2. Huber Loss (Smooth L1 Loss)
Used for: Robust regression, less sensitive to outliers than MSE


def huber_loss(y_true, y_pred, delta=1.0):
    """
    delta: threshold where loss transitions from quadratic to linear
    """
    error = y_true - y_pred
    abs_error = np.abs(error)
    
    quadratic = np.minimum(abs_error, delta)
    linear = abs_error - quadratic
    
    return 0.5 * quadratic**2 + delta * linear

# PyTorch implementation
class HuberLoss(nn.Module):
    def __init__(self, delta=1.0):
        super().__init__()
        self.delta = delta
    
    def forward(self, y_pred, y_true):
        error = y_true - y_pred
        abs_error = torch.abs(error)
        
        quadratic = torch.min(abs_error, torch.tensor(self.delta))
        linear = abs_error - quadratic
        
        loss = 0.5 * quadratic**2 + self.delta * linear
        return loss.mean()
3. Quantile Loss (Pinball Loss)
Used for: Quantile regression, uncertainty estimation


def quantile_loss(y_true, y_pred, quantile=0.5):
    """
    quantile: target quantile (0.5 for median)
    """
    error = y_true - y_pred
    loss = np.maximum(quantile * error, (quantile - 1) * error)
    return np.mean(loss)

# For multiple quantiles simultaneously
def multi_quantile_loss(y_true, y_preds, quantiles):
    """
    y_preds: predictions for each quantile [q1, q2, ...]
    """
    total_loss = 0
    for q, y_pred in zip(quantiles, y_preds):
        error = y_true - y_pred
        loss = np.maximum(q * error, (q - 1) * error)
        total_loss += np.mean(loss)
    return total_loss / len(quantiles)
4. Tweedie Loss
Used for: Compound Poisson-Gamma distributions (insurance claims, zero-inflated data)


def tweedie_loss(y_true, y_pred, p=1.5):
    """
    p: power parameter
    1 < p < 2: Compound Poisson-Gamma
    p = 1: Poisson
    p = 2: Gamma
    p = 3: Inverse Gaussian
    """
    # Stabilize predictions
    y_pred = np.clip(y_pred, 1e-10, None)
    
    if p == 1:
        # Poisson
        loss = y_pred - y_true * np.log(y_pred)
    elif p == 2:
        # Gamma
        loss = y_true / y_pred + np.log(y_pred)
    else:
        # General Tweedie
        loss = (y_pred**(2-p)) / (2-p) - (y_true * y_pred**(1-p)) / (1-p)
    
    return np.mean(loss)
5. Focal Loss
Used for: Class imbalance in object detection/classification


def focal_loss(y_true, y_pred, gamma=2.0, alpha=0.25):
    """
    gamma: focusing parameter (γ)
    alpha: balancing parameter
    """
    # Binary cross-entropy
    bce = - (y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    
    # Focal weight
    p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
    focal_weight = (1 - p_t) ** gamma
    
    # Alpha weighting
    alpha_weight = y_true * alpha + (1 - y_true) * (1 - alpha)
    
    loss = alpha_weight * focal_weight * bce
    return np.mean(loss)

# PyTorch implementation
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
    
    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return F_loss.mean()
6. Dice Loss (F1 Loss)
Used for: Image segmentation, imbalanced segmentation tasks


def dice_loss(y_true, y_pred, smooth=1e-6):
    """
    Dice coefficient = 2|X∩Y| / (|X| + |Y|)
    Dice Loss = 1 - Dice coefficient
    """
    # Flatten
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    
    intersection = np.sum(y_true_f * y_pred_f)
    union = np.sum(y_true_f) + np.sum(y_pred_f)
    
    dice = (2. * intersection + smooth) / (union + smooth)
    return 1 - dice

# With PyTorch
class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super().__init__()
        self.smooth = smooth
    
    def forward(self, y_pred, y_true):
        # y_pred: [B, C, H, W], y_true: [B, C, H, W]
        intersection = torch.sum(y_pred * y_true)
        union = torch.sum(y_pred) + torch.sum(y_true)
        dice = (2. * intersection + self.smooth) / (union + self.smooth)
        return 1 - dice
7. Triplet Loss
Used for: Metric learning, face recognition, Siamese networks


def triplet_loss(anchor, positive, negative, margin=1.0):
    """
    anchor, positive, negative: embeddings
    margin: minimum distance between positive and negative pairs
    """
    pos_dist = np.sum((anchor - positive)**2, axis=1)
    neg_dist = np.sum((anchor - negative)**2, axis=1)
    
    loss = np.maximum(0, pos_dist - neg_dist + margin)
    return np.mean(loss)

# PyTorch implementation
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super().__init__()
        self.margin = margin
    
    def forward(self, anchor, positive, negative):
        pos_dist = F.pairwise_distance(anchor, positive, p=2)
        neg_dist = F.pairwise_distance(anchor, negative, p=2)
        
        loss = torch.relu(pos_dist - neg_dist + self.margin)
        return loss.mean()
8. Contrastive Loss
Used for: Siamese networks, similarity learning


def contrastive_loss(y_true, distance, margin=1.0):
    """
    y_true: 1 for similar pairs, 0 for dissimilar
    distance: Euclidean distance between embeddings
    """
    # For similar pairs
    similar_loss = y_true * distance**2
    
    # For dissimilar pairs
    dissimilar_loss = (1 - y_true) * torch.maximum(margin - distance, 0)**2
    
    return torch.mean(similar_loss + dissimilar_loss) / 2

# Complete PyTorch implementation
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super().__init__()
        self.margin = margin
    
    def forward(self, output1, output2, label):
        # Euclidean distance
        euclidean_distance = F.pairwise_distance(output1, output2)
        
        loss_contrastive = torch.mean(
            (1 - label) * torch.pow(euclidean_distance, 2) +
            label * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2)
        )
        return loss_contrastive
Regression Algorithms
Linear Models
Linear Regression


from sklearn.linear_model import LinearRegression
model = LinearRegression()
Ridge Regression (L2 regularization)


from sklearn.linear_model import Ridge
model = Ridge(alpha=1.0)
Lasso Regression (L1 regularization)


from sklearn.linear_model import Lasso
model = Lasso(alpha=0.1)
ElasticNet (L1 + L2)


from sklearn.linear_model import ElasticNet
model = ElasticNet(alpha=1.0, l1_ratio=0.5)
Tree-based Models
Decision Tree Regressor


from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(max_depth=5)
Random Forest Regressor


from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100)
Gradient Boosting


from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3
)
XGBoost


import xgboost as xgb
model = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5
)
LightGBM


import lightgbm as lgb
model = lgb.LGBMRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=-1  # No limit
)
Kernel Methods
Support Vector Regression (SVR)


from sklearn.svm import SVR
model = SVR(kernel='rbf', C=1.0, epsilon=0.1)
Gaussian Process Regression


from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
kernel = 1.0 * RBF(length_scale=1.0)
model = GaussianProcessRegressor(kernel=kernel)
Neural Networks
MLP Regressor


from sklearn.neural_network import MLPRegressor
model = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    max_iter=1000
)
Deep Learning (TensorFlow/PyTorch)


# TensorFlow example
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer
])
Specialized Regressors
Quantile Regressor


from sklearn.linear_model import QuantileRegressor
model = QuantileRegressor(quantile=0.5, alpha=1.0)
Poisson Regressor (for count data)


from sklearn.linear_model import PoissonRegressor
model = PoissonRegressor(alpha=1.0)
Bayesian Ridge Regression


from sklearn.linear_model import BayesianRidge
model = BayesianRidge()
Ensemble Methods
Stacking Regressor


from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression

estimators = [
    ('rf', RandomForestRegressor()),
    ('xgb', xgb.XGBRegressor())
]
model = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearRegression()
)
Voting Regressor


from sklearn.ensemble import VotingRegressor

model = VotingRegressor([
    ('lr', LinearRegression()),
    ('rf', RandomForestRegressor()),
    ('xgb', xgb.XGBRegressor())
])